## Simple DNN on MNIST dataset

간단한 Pytorch 신경망 예제

#### Load Data

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

In [3]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

#### Define Model

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.main_layer = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.main_layer(x)
        return x

In [5]:
model = NeuralNetwork()

#### Train Model

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train() #모델을 train mode로 설정

    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            print(f'loss: {loss} [{batch*batch_size}/{len(dataloader.dataset)}]')


def test_loop(dataloader, model, loss_fn):
    model.eval() #모델을 evaluation mode로 설정
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad(): #gradient 계산 안 하도록 함
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*correct}%, Avg loss: {test_loss} \n")

In [9]:
for t in range(epochs):
    print(f"Epoch {t+1}")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
loss: 2.29947829246521 [0/60000]
loss: 0.590912401676178 [6400/60000]
loss: 0.4127355217933655 [12800/60000]
loss: 0.5082762837409973 [19200/60000]
loss: 0.4503677189350128 [25600/60000]
loss: 0.4467264413833618 [32000/60000]
loss: 0.36312004923820496 [38400/60000]
loss: 0.5310969352722168 [44800/60000]
loss: 0.4993456304073334 [51200/60000]
loss: 0.5272765159606934 [57600/60000]
Test Error: 
 Accuracy: 84.11%, Avg loss: 0.434192882316887 

Epoch 2
loss: 0.26273953914642334 [0/60000]
loss: 0.35331833362579346 [6400/60000]
loss: 0.276069313287735 [12800/60000]
loss: 0.3917500972747803 [19200/60000]
loss: 0.39774423837661743 [25600/60000]
loss: 0.3779125213623047 [32000/60000]
loss: 0.3158171474933624 [38400/60000]
loss: 0.4803829789161682 [44800/60000]
loss: 0.3958195745944977 [51200/60000]
loss: 0.45014166831970215 [57600/60000]
Test Error: 
 Accuracy: 85.5%, Avg loss: 0.39023692991323533 

Epoch 3
loss: 0.22171302139759064 [0/60000]
loss: 0.33228784799575806 [6400/60000]
loss:

#### Save Model

In [10]:
torch.save(model, './data/models/mnistDNN.pth')